Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Design Overview

The goal of this worksheet is to help you think about database design. Design is important because:

1. Sound databases allow for persistent storage of data, as a shared resource accessible to multiple users, in such a way that future insertions will not break pre-existing functions.
2. When databases are well-designed, it becomes easier to write code to interface with them.
3. 80% of a data scientist's time is spent reading, munging, and cleaning data. Doing these tasks up front, once, in order to get the data into a database, will save time downstream.


## Design process

The process of design breaks down into three steps:

1. Figure out the tables that should be part of your database. Each table should deal with logically independent units (e.g., students, classes, courses, instructors) and should have its own functional dependency (e.g., the difference between `countries` and `indicators`).
2. Figure out the fields of each table. Avoid fields that are a mashup, or whose value is a list. Avoid fields full of repeats (e.g., `land` mass as a field in `indicators` with repeated values every year because the landmass of a country doesn't change over time). Avoid repeated fields between tables unless used for a join (e.g., don't include `credithours` in both `courses` and `classes` but it's ok to include `classid` in both `classes` and `student_class`).
3. Figure out the relationships between tables. If there is a many-to-many relationship, you will need to introduce a linking table.


## Warmup

To get you warmed up, please answer the questions below.

**Q1:** Below are two instances of relations of a school. 

|studID|grade|gpa|
|--|-----|-----|
|10699|Freshman|3.6|
|62045|Senior|2.4|
|38599|Sophomore|3.1|
<center>Grades</center>


|first|last|phone|ID|
|-----|----|-----|--|
|Rene|Bonham|651-800-0941|38599|
|Mackenzie|Dow|612-683-5855|10699|
|Esme|Finley|612-044-2893|62045|
<center>Students</center>

Answer the following:
1. What are the attributes of each relation?
2. What are the tuples of each relation?
3. What is the relation schema for each relation?
4. What is the database schema?
5. What is another equivalent way to present each relation?

YOUR ANSWER HERE

**Q2:** Create the previous two relations of a school into two pandas dataframes. Then join the two dataframes.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Q3:** How many different ways are there to represent a relation instance if that instance has:
1. 3 attributes and 2 tuples?
2. 3 attributes and 4 tuples?
3. n attributes and m tuples?

YOUR ANSWER HERE

**Q4:** Describe the structure, constraints, and operations used in a data model, and the problems solved by using a database. 

YOUR ANSWER HERE

**Q5:** What are the types of keys in various relation? Define and give an example of each.

YOUR ANSWER HERE

## Design Case Study

Now it is time for you to create tables and analyze them using python. The example below walks you though how a data scientist might approach the problem of creating a database, especially the types of questions you should be asking yourself.

**Q6:** Design a database for the following dataset. You do not have to create the tables, just list the columns for each. Note which columns are keys. Give the functional dependencies.

|Player|Team|Position|Age|Division|BattingAv|
|------|----|--------|---|--------|---------|
|George|Wildcats|First base|50|West|0.316|
|Rob|Wildcats|Second base|42|West|0.197|
|Eric|Wildcats|Third base|28|West|0.388|
|Jim|Penguins|Pitcher|45|East|0.455|
|Greg|Penguins|Shortstop|38|East|0.215|
|Francis|Royals|Outfield|41|East|0.121|
|Moe|Raptors|Catcher|35|West|0.132|
|Edmund|Raptors|Pitcher|47|West|0.494|
|Curtis|Royals|First base|32|East|0.261|
|Dion|Wildcats|Catcher|46|West|0.207|
|Cecil|Royals|Second base|27|East|0.369|



YOUR ANSWER HERE

**Q7:** What types should be used for each field? Which fields should be allowed to take null values? Which fields should be primary keys? Are there any concerns about two different rows having the same value of the primary key? If so, how could this be fixed? Justify your answers.

YOUR ANSWER HERE

**Q8:** What type of relationship makes sense between `Players` and `Teams`? Which field should be a foreign key to facilitate efficient joining between these tables?

YOUR ANSWER HERE

**Q9:** What if this data represents an intramural league, where one player could play on multiple teams? What then would be the type of relationship between `Players` and `Teams`, and how would you facilitate this in your database. Be as precise as possible, e.g., discussing functional dependencies, foreign keys, joins, etc.

YOUR ANSWER HERE

**Q10:** Suppose now you learn that in the future, the data will be enriched by adding information about the rules in each `Division`, e.g., whether or not the division allows a "Designated Hitter" position. How does this change your database design from above?

YOUR ANSWER HERE

## Creating and populating tables

Now that we've thought through the database from a design point of view, we are almost ready to create a database for the data above. First, we need to set up a connection to our personal space in the database server. Please follow the instructions in the following cell.

### Set User Credentials

With a shared resource at a provider like a MySQL RDBMS, we need to use credentials to authenticate ourselves to the server, and need the logical location of the server itself.

For these notebooks, these are kept in a text file named 'creds.json', stored either in the same directory or in a data directory.  For this notebook, this is stored in the same directory as the notebook.

- Right click on the `creds.json` file and select *Open With*->*Editor*
- The server should be correct, mapped to `"hadoop2.mathsci.denison.edu"`. Likewise, the scheme should be correct, mapped to `"mysql+mysqlconnector"`.  
- Replace the mysql dictionary's key for "user" (currently `"nostudent"`) with the base part of your email address (i.e. without the `@denison.edu`).  Your password on the mysql server, at present, is the same as your username, so change that from `"nostudent"` as well.  
- **NEW INSTRUCTIONS** Lastly, change `database` from `"nostudent"` to your username, because you will be creating and dropping tables, and this must happen in your own space.

**Make sure to use double quotes for strings** ... this is `JSON`, not Python, and we have to follow JSON syntax.

Once this is complete, execute the following cell to connect to the database using SQL alchemy. If you are off-campus you will need to use a VPN first.

In [ ]:
import pandas as pd
import os
import os.path
import json
import sqlalchemy as sa

def getmysql_creds(dirname=".",filename="creds.json"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the four parts needed for a connection string to
        a remote provider using the "mysql" dictionary within
        an outer dictionary.  
        
        Return a scheme, server, user, and password
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    mysql = D["mysql"]
    return mysql["scheme"], mysql["server"], mysql["user"], mysql["pass"],mysql["database"]

scheme, server, u, password, database = getmysql_creds()
template = '{}://{}:{}@{}/{}'

cstring = template.format(scheme, u, password, server,database)

engine=sa.create_engine(cstring)

print(cstring) # you should be in your personal 
               # database space now, if you edited the JSON

**Q11:** Using sqlalchemy, add the tables you have created to your database. Then, print the tables you have created, using queries in SQL.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Q12:** Play around with select statements to get data out of your database you just created.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Updating a table

**Q13:** Let's say one year passed and the Wildcats traded George for Curtis. Make the respective changes to the database and then print the baseball table, ordered by team and then age.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Q14:** Using the `AVG()` query in SQL, create a view, called averages, that contains the average age and average batting average for each team, and print the result.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Q15:** Join `baseballAv` and teams but do not keep the Id column. Print the result.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## More Practice

The problems below give you even more practice.

**Question 1a:** Design a database for the following dataset. You do not have to create the tables, just list the columns for each. Note which columns are keys.



|name|continent|population|status|order|family|
|----|---------|----------|------|-----|------|
|polar bear|artic circle|20000|threatened|carnivora|ursidae|
|giant panda|asia|1000|endangered|carnivora|ursidae|
|american bison|north america|30000|near-endangered|artiodactyla|bovidae|
|cheetah|africa|7000|vulnerable|carnivora|felidae|
|south asian river dolphin|asia|1000|endangered|artiodactyla|platanistidae|
|snow leopard|asia|4000|endangered|carnivora|felidae|
|african lion|africa|30000|vulnerable|carnivora|felidae|
|indian rhinoceos|asia|2500|vulnerable|perissodactyla|rhinocerotidae|




YOUR ANSWER HERE

**Question 1b:** Create the previous database in MySQLWorkbench and paste your code below.

YOUR ANSWER HERE

**Question 1c:** Based on the the previous database, write the following queries in SQL:
1. Find the population of the cheetah.
2. Find the status of the snow leopard.
3. Find all the animals in the artiodactyla order.
4. Find all the animals who don't live in Asia, with a population over 10,000.
5. Find the populations of all the animals in the carnivora order.

YOUR ANSWER HERE

**Question 2a:** Design a database for the following dataset. You do not have to create the tables, just list the columns for each. Note which columns are keys.  A full version of the following can be found here: https://catalog.data.gov/dataset/school-location



|NAME|TYPE|PUBLIC/PRIVATE|SCHOOL DISTRICT|OPEN/CLOSED|CITY|ZIP|
|----|----|--------------|---------------|-----------|-------------|-------|
|Baton Rouge Montessori Kids|All Grades|Private	|Private|	O|	BATON ROUGE|	70810|
|Greenville Alternative at Beechwood|Middle|Public|East Baton Rouge Parish|O|BATON ROUGE|70807|
|Central Private School|All Grades|Private|Private|O|BAKER|70714|
|Baker Heights Elementary School|Elementary|Public|City of Baker|O|BAKER|70714|
|Northwestern Middle School|Middle|Public|Zachary Community|O|ZACHARY|70791|
|Zachary Early Learning Center|Elementary|Public|Zachary Community|O|ZACHARY|70791|
|Woodlawn High School|High|Public|East Baton Rouge Parish|O|BATON ROUGE|70817|
|Family Christian Academy|All Grades|Private|Private|O|BATON ROUGE|70810|
|Wildwood Elementary School|Elementary|Public|East Baton Rouge Parish|O|BATON ROUGE|70810|


YOUR ANSWER HERE

**Question 2b:** Insert the following SQL into MySQLWorkbench and paste your code below.

YOUR ANSWER HERE

**Question 2c:** Based on the the previous database, write the following queries in SQL:
1. Find all the schools in 70810. 
2. Find all the public middle schools.
3. Find all the schools in Baton Rouge. 
4. Find the public, all grade schools in Baton Rouge. 
5. Find all the schools with School in the name. 

YOUR ANSWER HERE

**Question 3a:** Design a database for the following dataset. You do not have to create the tables, just list the columns for each. Note which columns are keys.

|State Park|Course Name|Region|County|
|----------|-----------|------|------|
|Alfred E. Smith/Sunken Meadow|Blue|9|Suffolk|
|Alfred E. Smith/Sunken Meadow|Green|9|Suffolk|
|Alfred E. Smith/Sunken Meadow|Red|9|Suffolk|
|Battle Island||5|	Oswego|
|Beaver Island|	|	1|	Erie|
|Bethpage	|Black	|9|	Nassau|
|Bethpage	|Blue	|9|	Nassau|
|Bethpage	|Green	|9|	Nassau|
|Bethpage	|Red	|9|	Nassau|
|Bethpage	|Yellow	|9|	Nassau|
|Bonavista		||4|Seneca|
|Chenango Valley||		5|	Broome|
|Fair Haven Beach||		4|	Cayuga|
|Green Lakes	||	5	|Onondaga|
|Indian Hills	||	4	|Steuben|
|James Baird		||7	|Dutchess|
|Jones Beach	|Pitch/Putt	|9	|Nassau|
|Mark Twain		||4	|Chemung|
|Montauk Downs	||	9|	Suffolk|
|Ogden & Ruth Livingston Mills	||	7|	Dutchess|
|Pinnacle		||4	|Steuben|
|Robert Moses	|Pitch/Putt|	9|	Suffolk|
|Rockland Lake	||	8	|Rockland|
|Rockland Lake	|Executive|	8	|Rockland|
|Sag Harbor		||9|	Suffolk|
|Saratoga Spa		||11|	Saratoga|
|Saratoga Spa	|Executive|11|	Saratoga|
|St. Lawrence		||10|	St. Lawrence|
|Wellesley Island	||	10	|Jefferson|

YOUR ANSWER HERE

**Question 3b:** Insert the following SQL into MySQLWorkbench and paste your code below.

YOUR ANSWER HERE

**Question 3c:** Based on the the previous database, write the following queries in SQL:
1. Find the golf courses.
2. Find the golf courses in Suffolk.
3. Find all the State Parks which contain an s in their name.
4. Find the names of the blue courses in region 9. 

YOUR ANSWER HERE

**Question 4a:** Using pd.read_csv(), read in the following csv as df:https://vincentarelbundock.github.io/Rdatasets/csv/DAAG/spam7.csv.

Then show the first few rows of the dataframe.

For a description of the dataset, read the following: https://vincentarelbundock.github.io/Rdatasets/doc/DAAG/spam7.html

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Question 4b:** Using pandas, create a new dataframe containing the last 3 columns. Call this dataframe dfB.  Then write the equivalent query in SQL.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert list(dfB.iloc[5])==[0.0, 0.0, 'y']
assert list(dfB.iloc[234])==[0.0, 0.83999999999999997, 'y']
assert len(dfB)==4601

**Question 4c:** Using pandas, create a new dataframe containing of only the spam emails. Call this dataframe dfC. Then write the equivalent query in SQL.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert len(dfC)==1813

**Your answer here**

**Question 4d:** Using pandas, create a dataframe of the averages of each column, grouped by whether or not the email was spam. Call this dataframe dfD. Then write the equivalent query in SQL. **Hint:** to take the average in SQL, use AVG()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
assert list(dfD['dollar'])==[0.011648493543758971, 0.17447821290678439]

**Question 4e:** Using pandas, create a new dataframe that is sorted by total length of words in capitals, from greatest to least. Call this dataframe dfE. Then write the equivalent query in SQL.

In [ ]:
dfE=df.sort_values('crl.tot',ascending=False)
dfE[:10]

In [ ]:
assert list(dfE.loc[10])==[11, 21, 0.0, 0.46200000000000002, 0.0, 0.0, 0.0, 'y']
assert len(dfE)==4601

**Your answer here**

**Question 5a:** The following is simulated data representing forecasters weather predictions. Create the following table using SQL queries in MySQLWorkbench and paste your query below: 

|ID|x1|x2|x3|x4|
|--|--|--|--|--|
|0|Merle|64|67|Hibbing|
|1|Jillian|68|63|Duluth|
|2|Julia|62|71|Elk River|
|3|Paul|76|76|Eden Praire|
|4|Rob|70|74|Bloomington|
|5|Andrew|65|71|Park Rapids|
|6|Maya|65|67|Eagan|

**Your answer here**

**Question 5b:** Based on the previously generated table, write SQL queries that do the following:
1. Name the columns as follows: name, prediction, actual, location.
2. Delete Paul from the table.
3. Add the following row: ['George',80,74,Lakeville].
4. Create a new column, error1, that calculates how far off their predition was.
5. Changes George's prediction to 79.
6. Find who got the largest error.

**Your answer here**

**Question 6:** Write a function

    importCsv(url,tableName,cursor)

that reads in the following url to a database: https://vincentarelbundock.github.io/Rdatasets/csv/DAAG/cuckoos.csv. Information about the data can be found here: https://vincentarelbundock.github.io/Rdatasets/doc/DAAG/cuckoos.html.

In [ ]:
# Clean up the code below

import numpy as np
import pandas as pd

def tupleConvert(row):
    crow = []
    for item in row[1:]:
        if isinstance(item, str):
            crow.append(item)
        elif item == None:
            crow.append(item)
        elif item == float("nan"):
            crow.append(None)
        elif pd.isnull(item):
            crow.append(None)
        else:
            crow.append(np.asscalar(item))
    return crow

def importCsv(url,tableName,cursor):
    df=pd.read_csv(url)
    c=cursor
    c.execute("drop table if exists "+tableName)
    
    types=list(df.dtypes)
    types2=[]
    for item in types:
        item=str(item).replace('64','')
        if item=='object':
            item='text'
        types2.append(item)
    columns=[]
    for index in range(len(df.columns)):
        columns.append((df.columns[index].replace(' ','').replace(':',''),types2[index]))
    
    text1="create table "+tableName+"("
    for index in range(len(columns)):
        text1=text1+columns[index][0]+" "+columns[index][1]+', '
    text1=text1[:-2]+')'
    c.execute(text1)

    for row in df.itertuples():
            row=tupleConvert(row)
            text="insert into "+tableName+" values("+str(row[0])+','+str(row[1])+','+str(row[2])+",'"+str(str(row[3]))+"',"+\
              str(row[4])+")"
            c.execute(text)
c=engine.connect() 
url='https://vincentarelbundock.github.io/Rdatasets/csv/DAAG/cuckoos.csv'
importCsv(url,"cuck",c)
c.close()

In [ ]:
# Just in case the connection is still open, close it.
c.close()
